In [1]:
from paramiko.client import SSHClient
from paramiko import AutoAddPolicy
from setting import side_view_computer_left,side_view_computer_right,top_view_computer
import os
from time import sleep
def run_command_on_server(server_config,cmd):
    client = SSHClient()
    client.set_missing_host_key_policy(AutoAddPolicy())
    client.connect(server_config['ip'], username=server_config['user'], password=server_config['pwd'])
    stdin, stdout, stderr = client.exec_command(cmd)
    stdout, stderr = stdout.read().decode(), stderr.read().decode()
    client.close()
    return stdin, stdout, stderr

def get_animal_folders_from_server(server_config):
    stdin, folders, stderr = run_python_script(server_config,'list_available_animal_folders.py')
    folders =eval(folders)
    return folders

def get_trial_folders_from_server(server_config,animal_folder):
    stdin, folders, stderr = run_python_script(server_config,f'list_available_trial_folders.py --folder {animal_folder}')
    folders =eval(folders)  
    return folders

def run_python_script(server_config,python_script_command):
    return run_command_on_server(server_config,f"{server_config['dlc_environment']} && python {os.path.join(server_config['code_path'],python_script_command)}")

def process_folder_on_server(server_config,folder,trial):
    stdin, folders, stderr = run_python_script(server_config,f'process_experiment.py --folder "{folder}" --trial "{trial}"')
    print(folders)
    return stdin, folders, stderr

def pick_folder(folders,prompt):
    for i in range(len(folders)):
        print(f'{i+1}. {folders[i]}')
    sleep(0.01)
    folder = input(prompt)
    folder = folders[int(folder)-1]
    print(f'picked: {folder}')
    return folder

def start_remote_trials():
    computers = [side_view_computer_left,side_view_computer_right,top_view_computer]
    names = ['side_view_computer_left','side_view_computer_right','top_view_computer']
    computers = dict(zip(names,computers))
    for name,config in computers.items():
        print(name)
        files = get_animal_folders_from_server(side_view_computer_right)
        animal = pick_folder(files,'Pick an animal')
        trials = get_trial_folders_from_server(side_view_computer_right,animal)
        trial = pick_folder(trials,'Pick a trial')
        process_folder_on_server(config,animal,trial)

/usr/lib/python3/dist-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [5]:
get_animal_folders_from_server(top_view_computer)

['AR19distance',
 'ar21\x820-07-06',
 'ar30',
 'ar30ightduration',
 'ar30motor',
 'ar31',
 'ar32',
 'ar32MotorEye',
 'ar33',
 'ar33record',
 'ar34firstday',
 'ar34lightduration',
 'ar34motor',
 'ar34MotorEye',
 'ar34secondday',
 'ar35firstday',
 'ar35lightduration',
 'ar35motor',
 'ar35MotorEye',
 'ar35secondday',
 'ar38motor',
 'ratbehaviorinfo (AutoRecovered).rtf',
 'ratbehaviorinfo.rtf',
 '~$tbehaviorinfo (AutoRecovered).rtf',
 '~$tbehaviorinfo.rtf']